In [56]:
import pandas as pd
import numpy as np
from helper_functions.clean_transitions import clean_transition_column


In [4]:
transition_times = pd.read_excel('Transition_times_reduced.xlsx')

In [57]:
transition_times['Transition'] = clean_transition_column(transition_times['Transition'])

In [29]:
transition_matrix = pd.read_excel('first_transition_matrix.xlsx',index_col='Starting_State')

In [9]:
scoring = pd.read_excel('Transitions_and_scores_v2.xlsx')

In [58]:
scoring['Transition']=clean_transition_column(scoring['Transition'])

In [32]:
def choose_ending_state(state):
    # Get the row corresponding to the starting state
    transition_probs = transition_matrix.loc[state]
    
    # Choose an ending state based on the probabilities
    ending_state = np.random.choice(transition_probs.index, p=transition_probs.values)
    
    return ending_state

In [7]:
#game starts with team_a = 0, team_b = 0, time = 20*60

In [53]:
a_score = 0
b_score = 0
time_remaining = 20*60

In [12]:
starting_state = np.random.choice(['Ai0','Bi0'])

In [51]:
choose_ending_state('Ai0')

'Br0'

In [52]:
first_half_events = pd.DataFrame(columns=['Transition','Team_A','Team_B','Time'])

In [59]:
#first crack at real simulation - will be slowed by dataframe transition lookup - would be better to use each unique
#transition as a key with the value being the time distribution, but can switch later
current_state = np.random.choice(['Ai0','Bi0'])
while time_remaining >= 0:
    next_state = choose_ending_state(current_state)
    transition = (current_state, next_state)
    possible_times = transition_times[transition_times['Transition']==transition].copy()
    time_elapsed = np.random.choice(possible_times['Time'])

    time_remaining-=time_elapsed

    play_score = scoring[scoring['Transition']==transition].copy()
    play_score['Time'] = time_remaining
    first_half_events = pd.concat([first_half_events, play_score])
    current_state = next_state

In [71]:
# first_half_events['Team_A_Final'] = first_half_events['Team_A'].cumsum()
# first_half_events['Team_B_Final'] = first_half_events['Team_B'].cumsum()

In [64]:
first_half_events

,Transition,Team_A,Team_B,Time,Team_A_Final,Team_B_Final
44,"(Ai0, Bi3)",3,0,1191.0,3,0
35,"(Bi3, Ai2)",0,2,1173.0,3,2
36,"(Ai2, Br0)",0,0,1157.0,3,2
4,"(Br0, Ar0)",0,0,1140.0,3,2
5,"(Ar0, Bi3)",3,0,1128.0,6,2
...,...,...,...,...,...,...
3,"(Ar0, Br0)",0,0,55.0,45,40
4,"(Br0, Ar0)",0,0,44.0,45,40
18,"(Ar0, Bi2)",2,0,38.0,47,40
60,"(Bi2, Ai2)",0,2,18.0,47,42


In [65]:
second_half_events = pd.DataFrame(columns=['Transition','Team_A','Team_B','Time'])

In [67]:
current_state = np.random.choice(['Ai0','Bi0'])
time_remaining=20*60
while time_remaining >= 0:
    next_state = choose_ending_state(current_state)
    transition = (current_state, next_state)
    possible_times = transition_times[transition_times['Transition']==transition].copy()
    time_elapsed = np.random.choice(possible_times['Time'])

    time_remaining-=time_elapsed

    play_score = scoring[scoring['Transition']==transition].copy()
    play_score['Time'] = time_remaining
    second_half_events = pd.concat([second_half_events, play_score])
    current_state = next_state

In [68]:
first_half_events['Period']=1
second_half_events['Period']=2

In [69]:
final_df = pd.concat([first_half_events,second_half_events])

In [72]:
final_df['Team_A_Final'] = final_df['Team_A'].cumsum()
final_df['Team_B_Final'] = final_df['Team_B'].cumsum()

In [73]:
final_df

,Transition,Team_A,Team_B,Time,Team_A_Final,Team_B_Final,Period
44,"(Ai0, Bi3)",3,0,1191.0,3,0,1
35,"(Bi3, Ai2)",0,2,1173.0,3,2,1
36,"(Ai2, Br0)",0,0,1157.0,3,2,1
4,"(Br0, Ar0)",0,0,1140.0,3,2,1
5,"(Ar0, Bi3)",3,0,1128.0,6,2,1
...,...,...,...,...,...,...,...
15,"(Af1, Bi1)",1,0,58,84,87,2
40,"(Bi1, Ar0)",0,0,35,84,87,2
18,"(Ar0, Bi2)",2,0,25,86,87,2
60,"(Bi2, Ai2)",0,2,10,86,89,2


In [74]:
#very happy with the first result of 89-88

In [77]:
first_half_events = pd.DataFrame(columns=['Transition','Team_A','Team_B','Time'])
time_remaining=20*60
#first crack at real simulation - will be slowed by dataframe transition lookup - would be better to use each unique
#transition as a key with the value being the time distribution, but can switch later
current_state = np.random.choice(['Ai0','Bi0'])
while time_remaining >= 0:
    next_state = choose_ending_state(current_state)
    transition = (current_state, next_state)
    possible_times = transition_times[transition_times['Transition']==transition].copy()
    time_elapsed = np.random.choice(possible_times['Time'])

    time_remaining-=time_elapsed

    play_score = scoring[scoring['Transition']==transition].copy()
    play_score['Time'] = time_remaining
    first_half_events = pd.concat([first_half_events, play_score])
    current_state = next_state

first_half_events['Team_A_Final'] = first_half_events['Team_A'].cumsum()
first_half_events['Team_B_Final'] = first_half_events['Team_B'].cumsum()

print('First Half Result: ')
print(first_half_events.tail(1))

second_half_events=pd.DataFrame(columns=['Transition','Team_A','Team_B','Time'])

current_state = np.random.choice(['Ai0','Bi0'])
time_remaining=20*60
while time_remaining >= 0:
    next_state = choose_ending_state(current_state)
    transition = (current_state, next_state)
    possible_times = transition_times[transition_times['Transition']==transition].copy()
    time_elapsed = np.random.choice(possible_times['Time'])

    time_remaining-=time_elapsed

    play_score = scoring[scoring['Transition']==transition].copy()
    play_score['Time'] = time_remaining
    second_half_events = pd.concat([second_half_events, play_score])
    current_state = next_state

first_half_events['Period']=1
second_half_events['Period']=2

final_df = pd.concat([first_half_events,second_half_events])

final_df['Team_A_Final'] = final_df['Team_A'].cumsum()
final_df['Team_B_Final'] = final_df['Team_B'].cumsum()

First Half Result: 
    Transition Team_A Team_B Time Team_A_Final Team_B_Final
53  (Bi1, Bi0)      0      0   -5           42           41


In [78]:
final_df

,Transition,Team_A,Team_B,Time,Team_A_Final,Team_B_Final,Period
37,"(Bi0, Ai2)",0,2,1183,0,2,1
61,"(Ai2, Bi3)",3,0,1161,3,2,1
35,"(Bi3, Ai2)",0,2,1137,3,4,1
83,"(Ai2, Ai0)",0,0,1125,3,4,1
29,"(Ai0, Br0)",0,0,1114,3,4,1
...,...,...,...,...,...,...,...
15,"(Af1, Bi1)",1,0,54,80,80,2
40,"(Bi1, Ar0)",0,0,30,80,80,2
2,"(Ar0, Ar0)",0,0,23,80,80,2
5,"(Ar0, Bi3)",3,0,18,83,80,2
